# 1. Imports

In [7]:
import numpy as np
from math import pi

In [3]:
from tensorflow.keras.models import load_model

In [4]:
# import the necessary packages
from tensorflow.keras.losses import MSE
from tensorflow.keras.losses import categorical_crossentropy as CCE
import tensorflow as tf

In [10]:
path_model = "ACAS_XU_tf_keras/ACASXU_1_1.h5"

# 2. Load models

In [11]:
model_11 = load_model(path_model)

In [16]:
model_35 = load_model("ACAS_XU_tf_keras/ACASXU_3_5.h5")

In [12]:
model_11.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_98 (Dense)             (None, 50)                300       
_________________________________________________________________
activation_98 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_99 (Dense)             (None, 50)                2550      
_________________________________________________________________
activation_99 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_100 (Dense)            (None, 50)                2550      
_________________________________________________________________
activation_100 (Activation)  (None, 50)                0         
_________________________________________________________________
dense_101 (Dense)            (None, 50)              

In [13]:
model_11.compile()

In [17]:
model_35.compile()

In [27]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_11.predict(pt)

array([[0.19883643, 0.20087564, 0.19964513, 0.20093843, 0.19970441]],
      dtype=float32)

In [18]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_35.predict(pt)

array([[0.1967246 , 0.2045978 , 0.19686562, 0.20457308, 0.19723897]],
      dtype=float32)

# 3. FGSM

In [58]:
def generate_adv_sample(model, x0, label, loss_function="MSE", eps=1e-5):
    # transforming into a tensorflow object
    x0_ = tf.cast(x0, tf.float32)
    
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our input should be tacked for gradient updates
        tape.watch(x0_)

        # use our model to make predictions on the input and then compute the loss
        pred = model(x0_)
        if loss_function == "CCE":
            np_label = np.array([i==label for i in range(0,5)]).reshape((1,5))
            loss = CCE(np_label, pred)
        elif loss_function == "MSE":
            loss = MSE(label, pred)
        else:
            raise Exception("Unknown loss function '{0}'".format(loss_function))
        
        # calculate the gradients of loss with respect to the input, then compute the sign of the gradient
        gradient = tape.gradient(loss, x0_)
        signedGrad = tf.sign(gradient)

        # construct the image adversary
        adv_sample = (x0_ + (signedGrad * eps)).numpy()

        # return the adversarial sample to the calling function
        return(adv_sample)

In [62]:
x0_adv = generate_adv_sample(model_11, pt, label = 0, loss_function = "CCE", eps=1e-3)
print(x0_adv)

[[0.999 1.001 0.999 1.001 0.999]]


In [38]:
model_11.predict(x0_adv)

array([[0.19881836, 0.20089944, 0.19962737, 0.20096485, 0.19969004]],
      dtype=float32)

In [55]:
np_label = np.array([int(i==2) for i in range(0,5)]).reshape((1,5))
np_label

array([[0, 0, 1, 0, 0]])

# 4. Properties

In [8]:
# PROCEDURE : X_normalised = (X - X_mean)/X_range
# X = [rho, theta, psi, v_own, v_int] (in that order)

X_mean = np.array( [1.9791091*10e4, 0.0, 0.0, 650.0, 600.0] )
X_range = np.array( [60261.0, 6.28318530718, 6.28318530718, 1100.0, 1200.0] )

X_dom = np.array([ [0.0,    -pi, -pi, 100.0,     0.0],
                   [60760.0, pi,  pi, 1200.0, 1200.0] ])

# u = np.true_divide(x - X_mean, X_range) # element-wise division

In [10]:
# PROPERTY 1

def IP1(x):
    """ Check if inputs of property 1 are satisfied. """
    return( (x[0] >= 55947.691) and (x[3] >= 1145) and (x[4] <= 60) )
    
def OP1(x, coc_mean=7.5188840201005975, coc_range=373.94992):
    """ Check if outputs of property 1 are satisfied. """
    coc = (x[0] - coc_mean)/coc_range
    return(coc <= 1500)


# PROPERTY 2

def IP2(x):
    return( (x[0] >= 55947.691) and (x[3] >= 1145) and (x[4] <= 60) )

def OP2(x):
    """ COC score is not the maximal score. """
    return(np.argmax(x) != 0)


# PROPERTY 3

def IP3(x):
    return( (1500.0 <= x[0] <= 1800.0) and (-0.06 <= x[1] <= 0.06) and (3.10 <= x[2]) and (980.0 <= x[3]) and (960.0 <= x[4]) )

def OP3(x):
    """ COC score is not the minimal score. """
    return(np.argmin(x) != 0)


# PROPERTY 4

def IP4(x):
    return( (1500.0 <= x[0] <= 1800.0) and (-0.06 <= x[1] <= 0.06) and (x[2] == 0) and (1000.0 <= x[3]) and (700.0 <= x[4] <= 800.0) )

def OP4(x):
    """ COC score is not the minimal score. """
    return(np.argmin(x) != 0)


# PROPERTY 5

def IP5(x):
    return( (250.0 <= x[0] <= 400.0) and (0.2 <= x[1] <= 0.4) and (-3.141592 <= x[2] <= -3.141592 + 0.005) and (100.0 <= x[3] <= 400.0) and (0.0 <= x[4] <= 400.0) )

def OP5(x):
    """ SRC score is the minimal score. """
    return(np.argmin(x) == 3)


# PROPERTY 6

def IP6(x):
    return( (12000.0 <= x[0] <= 62000.0) and ((0.7 <= x[1] <= 3.141592) or (-3.141592 <= x[1] <= -0.7)) and (-3.141592 <= x[2] <= -3.141592+0.005) and (100.0 <= x[3] <= 1200.0) and (0.0 <= x[4] <= 1200.0) )

def OP6(x):
    """ COC score is the minimal score. """
    return(np.argmin(x) == 0)


# PROPERTY 7

def IP7(x):
    return( (12000.0 <= x[0] <= 60760.0) and (-3.141592 <= x[1] <= 3.141592) and (-3.141592 <= x[2] <= 3.141592) and (100.0 <= x[3] <= 1200.0) and (0.0 <= x[4] <= 1200.0) )

def OP7(x):
    """ SR score and SL score are not the minimal scores. """
    min_score_ind = np.argmin(x)
    return( (min_score_ind != 3) and (min_score_ind != 4) )


# PROPERTY 8

def IP8(x):
    return( (0.0 <= x[0] <= 60760.0) and (-3.141592 <= x[1] <= -0.75*3.141592) and (-0.1 <= x[2] <= 0.1) and (600.0 <= x[3] <= 1200.0) and (600.0 <= x[4] <= 1200.0) )

def OP8(x):
    """ COC score or WL score is the minimal score. """
    min_score_ind = np.argmin(x)
    return( (min_score_ind == 0) and (min_score_ind == 2) )


# PROPERTY 9

def IP9(x):
    return( (2000.0 <= x[0] <= 7000.0) and (-0.4 <= x[1] <= -0.14) and (-3.141592 <= x[2] <= -3.141592+0.01) and (100.0 <= x[3] <= 150.0) and (0.0 <= x[4] <= 150.0) )

def OP9(x):
    """ SL score is the minimal score. """
    return(np.argmin(x) == 4)


# PROPERTY 10

def IP10(x):
    return( (36000.0 <= x[0] <= 60760.0) and (0.7 <= x[1] <= 3.141592) and (-3.141592 <= x[2] <= -3.141592+0.01) and (900.0 <= x[3] <= 1200.0) and (600.0 <= x[4] <= 1200.0) )

def OP10(x):
    """ COC score is the minimal score. """
    return(np.argmin(x) == 0)

# 5. Random search